In [ ]:
%matplotlib notebook

import SimpleITK as sitk
#import itkwidgets
import os, glob, re, math, sys

In [ ]:
import filebrowser

In [ ]:
f=filebrowser.FileBrowser()
print("Select an image from a series")
f.widget()

In [ ]:
# strip off the last 7 characters (###.tif) and replace with *.tif
search_string = f.path[:-7]+'*.tif'
search_string

In [ ]:
file_names = glob.glob(search_string)
file_names.sort()
print (file_names[0])
print ("  ...")
print (file_names[len(file_names)-1])
print(len(file_names), "files")

In [ ]:
# Read the header of the first image to get image size
reader = sitk.ImageFileReader()
reader.SetFileName(file_names[0])
reader.ReadImageInformation()
size = reader.GetSize()
bin_amount = int(size[0]/1000+0.5)
print("Image width =", size[0])
print("Binning by", bin_amount)

In [ ]:
#images are shrunk to less than 1000x1000
slices = []
n = len(file_names)
#n = 40

for i in range(n):
    try:
        slices.append(sitk.BinShrink(sitk.ReadImage(file_names[i]),[bin_amount,bin_amount]))
        print (i, end=' ')
    except:
        print("\nError: couldn't read file", file_names[i])
        break

In [ ]:
print (file_names)

In [ ]:
import gui
v = sitk.JoinSeries(slices)

acquire_centers = gui.PointDataAquisition(v)

In [ ]:
import itkwidgets
itkwidgets.view(v, mode='z', cmap='Grayscale')

In [ ]:
def segmentBackground(img):
    """ segment the image background.
    
    It does this by thesholding, applying a median filter,
    eroding, and selecting the largest connected component.
    """
    thresh = sitk.Median(sitk.BinaryThreshold(img, 15000, 36000),[2,2])
    erode = sitk.BinaryErode(thresh,[1,1])
    # selects the largest connected component
    connected = sitk.Cast(sitk.ConnectedComponent(erode), sitk.sitkUInt8)==1
    return connected

In [ ]:
bg=[]
for s in slices:
    bg.append(segmentBackground(s))
    
v = sitk.JoinSeries(bg) * 255
itkwidgets.view(v, mode='z', cmap='Grayscale')

In [ ]:
def computeExtents(img, center, radius):
    """ Compute the extents of a region in index space given its center and radius in physical space """
    # corners in physical space
    mincorner = [center[0]-radius, center[1]-radius]
    maxcorner = [center[0]+radius, center[1]+radius]
    
    minindex = list(img.TransformPhysicalPointToIndex(mincorner))
    maxindex = list(img.TransformPhysicalPointToIndex(maxcorner))
    size = img.GetSize()
    for i in range(2):
        if minindex[i]<0:
            minindex[i]=0
        if maxindex[i] >= size[i]:
            maxindex[i]=size[i]-1
    return minindex, maxindex

def findEllipse(img, center, radius):
    """ Find an ellipse given an approximate center and radius in physical space """
    minindex, maxindex = computeExtents(img, center, radius)
    #sitk.Show(img)
    cropped = img[minindex[0]:maxindex[0], minindex[1]:maxindex[1]]
    #sitk.Show(cropped)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(cropped)
    if stats.GetNumberOfLabels() == 0:
        print("Uh-oh: no labels")
        print (minindex, maxindex)
        try:
            sitk.Show(cropped)
        except:
            print("No show")
        return None, None
    c = stats.GetCentroid(1)
    axes = stats.GetPrincipalAxes(1)
    moments = stats.GetPrincipalMoments(1)
    #print("Center:", c)
    #print("Axes:", axes)
    return c, axes

In [ ]:
seeds=[]
picked_seeds = acquire_centers.get_point_indexes()
if len(picked_seeds) != 2:
    print("Warning: there should be two seed points")
    if picked_seeds[0][2] != picked_seeds[1][2]:
        print("Warning: the two seeds are on different slices, using the larger of the two.")
else:
    print(picked_seeds)

    # transform the picked seeds to physical space
    first_z = max(picked_seeds[0][2], picked_seeds[1][2])
    first_bg = bg[first_z]
    for seed in picked_seeds:
        seed_tf = first_bg.TransformIndexToPhysicalPoint(seed[0:2])
        seeds.append(seed_tf)
    print(seeds)

In [ ]:
# Locations of the two holes in physical space
# These seeds sets are used if the interactive seed picking above didn't happen.

# seeds for initial test data set: /Slice & View 01-31/Images/SEM Image
if len(seeds) == 0:
    seeds=[[962,1641],[5293,1660]]

# seeds for 2/18 data set: Slice & View 02-18/Images/SEM Multi-Detector Image
if len(seeds) == 0:
    seeds = [ [678, 216], [618,3456]]

In [ ]:
radius = 500

all_ellipses = []

for seed in seeds:
    s = seed
    z = 0
    
    # loop through all the background images
    for b in bg:
        # invert the background image to get the hole label image
        label_img = 1-b
        center, axes = findEllipse(label_img, s, radius)
        
        if center==None:
            print("Failed on seed", seed, "Z=",z)
            try:
                sitk.Show(label_img)
            except:
                print("No show")
            break
        
        # add Z coord for 3d ellipse
        cz = [center[0], center[1], z]
        axis1 = [axes[0], axes[1], 0]
        axis2 = [axes[2], axes[3], 0]
        ellipse = [cz, axis1, axis2]
        all_ellipses.append(ellipse)

        # set this ellipse center as the seed for the next Z image
        s = center
        z=z+1

print(all_ellipses)

In [ ]:
import numpy as np
def ellipses2pointset(elist):
    """ convert ellipse centers to a numpy array """
    centers = []
    for e in elist:
        c = e[0]
        centers.append(c)
    np_points = np.array(centers)
    return np_points

In [ ]:
pts = ellipses2pointset(all_ellipses)
#print(pts)

In [ ]:
itkwidgets.view(v, mode='z', cmap='Grayscale',point_sets=[pts])

In [ ]:
import ipywidgets as widgets
w = widgets.Text(
    value='',
    description='Output file'
)
display(w)

In [ ]:
print (w.value)

In [ ]:
import math
# Write the results out
if len(w.value) == 0:
    outfile = sys.stdout
else:
    outfile = open(w.value, 'w')
print("Centers", file=outfile)
for e in all_ellipses:
    center = e[0]
    print("%8.2f %8.2f %8.2f" % (center[0], center[1], center[2]), file=outfile)
    
print("Inter Hole Distance", file=outfile)
distances=[]
# this assumes that there are 2 seeds points resulting in pairs of ellipses
n = int(len(all_ellipses)/2)
for i in range(n):
    e1 = all_ellipses[i]
    e2 = all_ellipses[i+n]
    dx = e1[0][0]-e2[0][0]
    dy = e1[0][1]-e2[0][1]
    d = math.sqrt(dx*dx + dy*dy)
    distances.append(d)
    print("%8.2f" % (d), file=outfile)

# change in distance through Z
print("Change in IHD through Z", file=outfile)
dd_dz = []
for i in range(n-1):
    dz = distances[i+1]-distances[i]
    dd_dz.append(dz)
    print("%8.2f" % (dz), file=outfile)
    
if outfile != sys.stdout:
    outfile.close()